# Paper - Example 1

## Initialization

In [ ]:
# Import the pyKasso package
import pykasso as pk

In [ ]:
# Import the pyKasso package
import pykasso as pk

# Create a pyKasso application
app = pk.pykasso()

# Declare the grid parameters
grid_parameters = {
    'x0': 0,
    'y0': 0,
    'z0': 0,
    'nx': 300,
    'ny': 300,
    'nz': 1,
    'dx': 10,
    'dy': 10,
    'dz': 10,
}

# Declare the project parameters
app.new_project(name='example_01', grid_parameters=grid_parameters)

In [ ]:
# Declare the model parameters
model_parameters = {
    'sks' : {},
    'outlets' : {
        'data' : [[2990, 1500]],
        'number' : 1,
        # 'number' : 2,
        # 'importance' : [1], # ERREUR - TODO
    },
    'inlets' : {
        'data' : [[0, 1500]],
        'number': 1,
        # 'number': 8,
        # 'importance' : [1, 1], # ERREUR - TODO
        # 'per_outlet' : [1, 3] # ERREUR - TODO
    },
    'fractures' : {
        'generate': {
            'family_01': {
                'density'      : 0.00005,
                'orientation'  : 135,
                'dip'          : 90,
                'length'       : 300,
            },
            'family_02': {
                'density'      : 0.00005,
                'orientation'  : 45,
                'dip'          : 90,
                'length'       : 500,
            }
        }
    }
}

## Compute

In [ ]:
# Compute 100 simulation in a row
for i in range(1):
    model_parameters['sks']['seed'] = i
    app.model.generate(model_parameters=model_parameters)

In [ ]:
# app.model.iterations

In [ ]:
# app.model._outlets

In [ ]:
# app.model.outlets

In [ ]:
# app.model.conceptual_model_table

## Plot results

In [ ]:
# Plot the last simulated karstic conduit network
f01 = app.visualizer.mpl_plot_2D(feature='karst',
                                 imshow_options={'cmap': 'binary'},
                                 scatter_inlets_options={},
                                 scatter_outlets_options={},
                                 )
f01.fig.savefig('fig/example_01_karst', bbox_inches='tight')
f01.fig

In [ ]:
# Plot the fracturation model
f02 = app.visualizer.mpl_plot_2D(feature='fractures',
                                 imshow_options={'cmap': 'binary'},
                                 )
f02.fig.savefig('fig/example_01_fracturation', bbox_inches='tight')
f02.fig

In [ ]:
# Plot the cost array
f03 = app.visualizer.mpl_plot_2D(feature='cost',
                                 imshow_options={'cmap': 'binary'},
                                 )
f03.fig.savefig('fig/example_01_cost', bbox_inches='tight')
f03.fig

In [ ]:
# Plot the time travel array
f04 = app.visualizer.mpl_plot_2D(feature='time',
                                 imshow_options={'cmap': 'binary'},
                                 )
f04.fig.savefig('fig/example_01_time', bbox_inches='tight')
f04.fig

In [ ]:
# Compute the mean of all the simulated karstic networks
mean = app.analyzer.compute_stats_on_networks('mean')
extent = app.project.grid.extent
imshow_options = {
    'extent' : extent,
    'cmap' : 'binary',
    'vmin' : 0,
    'vmax' : 0.1
}
f05 = app.visualizer.mpl_plot_array_2D(array=mean,
                                       imshow_options=imshow_options
                                       )
f05.fig.savefig('fig/example_01_mean_karstic_network', bbox_inches='tight')
f05.fig